# Querying a SQL DB

We can replicate our SQLDatabaseChain with Runnables.

In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
model_config_path = os.path.abspath(os.path.join('../custom_llms/'))
sys.path.insert(0, module_path)
sys.path.insert(0, model_config_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [2]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [3]:
from langchain.utilities import SQLDatabase

We’ll need the Chinook sample DB for this example. There’s many places to download it from, e.g. https://database.guide/2-sample-databases-sqlite/

In [4]:
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")

In [5]:
def get_schema(_):
    return db.get_table_info()

In [6]:
def run_query(query):
    return db.run(query)

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [8]:
sql_response.invoke({"question": "How many employees are there?"})

'SELECT COUNT(*) as NumberOfEmployees\\nFROM Employee;'

In [9]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [10]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
)

In [11]:
full_chain.invoke({"question": "How many employees are there?"})

'There are 8 employees in the database.'